# Setup

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


# Load data

In [2]:
dataset = pd.read_csv("./product_reviews.csv")
dataset

,review,label
0,beklentimin altında bir ürün kaliteli değil,neg
1,3. kademe hız da motor titreme yapıyor.bu sebe...,neg
2,başlığı sabit durmuyor. arka koruma demiri üfl...,neg
3,ürün gayet kalitesiz öğrenci evinde aldığımız ...,neg
4,yuksek hizda kullanirken titredim yaptigi icin...,neg
...,...,...
7979,ürün pahalı ama gerçekten farklı.,pos
7980,3 bıçaklı fiyatına 5 bıçak ve jel geldi. teşek...,pos
7981,piyasa fiyatının oldukça altında ve toplamda 9...,pos
7982,askerde hayat kurtarır abiler,pos


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7984 entries, 0 to 7983
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  7984 non-null   object
 1   label   7984 non-null   object
dtypes: object(2)
memory usage: 124.9+ KB


In [4]:
dataset["label"].value_counts()

pos    3998
neg    3986
Name: label, dtype: int64

# Data cleaning

In [5]:
import re
import nltk
from nltk.stem.snowball import SnowballStemmer

nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Polat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from snowballstemmer import TurkishStemmer

ts = TurkishStemmer()
clean_reviws = []

for review in dataset["review"]:
  review = re.sub("[\W || \d]", " ", review)
  review = review.lower()
  review = review.split()
  review = [word for word in review if word not in set(stopwords.words("turkish"))]
  review = " ".join(review)
  clean_reviws.append(review)

dataset["clean_reviws"] = clean_reviws

C:\Users\Polat\AppData\Local\Temp\ipykernel_1032\2720343626.py:7: FutureWarning: Possible set union at position 4
  review = re.sub("[\W || \d]", " ", review)


In [7]:
dataset

,review,label,clean_reviws
0,beklentimin altında bir ürün kaliteli değil,neg,beklentimin altında bir ürün kaliteli değil
1,3. kademe hız da motor titreme yapıyor.bu sebe...,neg,kademe hız motor titreme yapıyor sebebten kade...
2,başlığı sabit durmuyor. arka koruma demiri üfl...,neg,başlığı sabit durmuyor arka koruma demiri üfle...
3,ürün gayet kalitesiz öğrenci evinde aldığımız ...,neg,ürün gayet kalitesiz öğrenci evinde aldığımız ...
4,yuksek hizda kullanirken titredim yaptigi icin...,neg,yuksek hizda kullanirken titredim yaptigi icin...
...,...,...,...
7979,ürün pahalı ama gerçekten farklı.,pos,ürün pahalı gerçekten farklı
7980,3 bıçaklı fiyatına 5 bıçak ve jel geldi. teşek...,pos,bıçaklı fiyatına bıçak jel geldi teşekkürler h...
7981,piyasa fiyatının oldukça altında ve toplamda 9...,pos,piyasa fiyatının oldukça altında toplamda jile...
7982,askerde hayat kurtarır abiler,pos,askerde hayat kurtarır abiler


# Data preprocessing

In [8]:
X = dataset["clean_reviws"].values.astype("U")    # "U" for Unicode string
y = dataset["label"].values.astype("U")     # "U" for Unicode string

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)

(5588,)
(2396,)


## Count Vectorization

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()

X_train_count = count_vect.fit_transform(X_train)
X_test_count = count_vect.transform(X_test)

In [11]:
print(X_train_count.shape)
print(X_test_count.shape)

(5588, 18243)
(2396, 18243)


## TF*IDF transformation

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_trans = TfidfTransformer()

X_train_tfidf = tfidf_trans.fit_transform(X_train_count)
X_test_tfidf = tfidf_trans.transform(X_test_count)

In [13]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(5588, 18243)
(2396, 18243)


# Create model

In [14]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train)

MultinomialNB()

In [15]:
scores = cross_val_score(mnb, X_train_tfidf, y_train, cv=5)

print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard deviation:", scores.std())

Scores: [0.89177102 0.90250447 0.89624329 0.89256938 0.88182632]
Mean: 0.8929828972634659
Standard deviation: 0.006743877353536977


In [16]:
y_pred = mnb.predict(X_test_tfidf)
cm = confusion_matrix(y_test, y_pred)

print(cm)
print(classification_report(y_test, y_pred))

[[1049  179]
 [ 102 1066]]
              precision    recall  f1-score   support

         neg       0.91      0.85      0.88      1228
         pos       0.86      0.91      0.88      1168

    accuracy                           0.88      2396
   macro avg       0.88      0.88      0.88      2396
weighted avg       0.88      0.88      0.88      2396



# Predictions

In [17]:
for review, sentiment in zip(X_test[:5], y_pred[:]):
  print("{} => {}".format(review, sentiment))

mükemmel => pos
fiyata ancak kadar olur yorumlarda orjinal yazan arkadaşlar bilmediğiniz konularda yorum yapmayın milletin kafasını karıştırmayın zaten başlıkta orjinal ibaresi yok => neg
gün içinde elimde oldu => pos
ürünü ilk önce köpüksüz kullandım jilet etkisi vermedi nci kullanışımda köpüklü kullandım fena değil fakat söylenilen sinek kaydı etkisi filan yaratmıyor iyi almanız bastırmanız gerekiyor buda inanılmaz tahrişe yol açıyor açıkçası tekrar kullanmaya korktum temizleme ünitesi güzel fakat gece açtıysanız ses rahatsız eder => neg
olumsuzlukların dışında fiyatı yapacağı işe göre gayet iyi montaj tamamlandığında sağlam bir duruşu var ben içerisine kutularıyla birlikte eyşalarımı kılıflarıyla beraber bir kaç takım elbisemi koymak aldım kutu kılıfa vurgu yapmak istedim kadar dikkat ederseniz edin içerisine toz biriktirecek bir yapısı var saygılar => neg
